<a href="https://colab.research.google.com/github/AndrewGavluk/PythonML/blob/main/skillfactory_dl_2_screencast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF в Colab

In [ ]:
%config IPCompleter.greedy=True  # более лучший автокомплит

In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.14.0


In [ ]:
tf.reset_default_graph()
a = tf.placeholder(tf.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b

In [ ]:
print(c)

Tensor("matmul:0", shape=(2, 2), dtype=float32)


In [ ]:
print(list(tf.get_default_graph().get_operations()[-1].inputs))
print(list(tf.get_default_graph().get_operations()[-1].outputs))

[<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>, <tf.Tensor 'Variable/read:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'matmul:0' shape=(2, 2) dtype=float32>]


In [ ]:
s = tf.InteractiveSession()

In [ ]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

array([[2., 2.],
       [2., 2.]], dtype=float32)

In [ ]:
s.close()

# Оптимизация

In [ ]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])

In [ ]:
tf.trainable_variables()

In [ ]:
with tf.Session() as s:  # сессия сама закроется
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

1.0783744 1.8170177
0.8626995 1.1628913
0.6901596 0.7442504
0.5521277 0.4763203
0.4417022 0.304845
0.35336176 0.19510083
0.2826894 0.12486453
0.22615151 0.079913296
0.18092121 0.051144507
0.14473698 0.032732487


# TensorBoard логирование

In [ ]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [ ]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [ ]:
with tf.Session() as s:
  summary_writer = tf.summary.FileWriter("logs/1", s.graph)
  s.run(tf.global_variables_initializer())
  for i in range(10):
      _, curr_summaries = s.run([step, summaries])
      summary_writer.add_summary(curr_summaries, i)
      summary_writer.flush()

Локально можно запустить  `tensorboard --logdir=./logs` в консоли.

А вот так можно запустить прямо в Google Colab:

In [ ]:
! wget https://raw.githubusercontent.com/ZEMUSHKA/skillfactory-dl/master/setup_google_colab.py -O setup_google_colab.py -q
import setup_google_colab

# запускаем сервер tensorboard в фоне
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# открываем порт из Google Colab
setup_google_colab.expose_port_on_colab(6006)

# Обучение линейной модели

In [ ]:
# модельные данные
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.98432495 0.21895209 0.86283045]]


In [ ]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D), name="features")
target = tf.placeholder(tf.float32, shape=(None, 1), name="target")

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights


loss = tf.reduce_mean((target - predictions) ** 2, name="loss")

print(target.shape, predictions.shape, loss.shape)

(?, 1) (?, 1) ()


In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [ ]:
# производные это часть графа
tf.get_default_graph().get_operations()

In [ ]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss)

7.2150946
0.04585571
0.038876403
0.037462875
0.03717636
0.03711824


In [ ]:
# результат обучения
curr_weights.T

array([[0.79846495, 0.57642007, 0.56647074]], dtype=float32)

In [ ]:
# правильные веса
w.T

array([[0.7692187 , 0.59123648, 0.56121469]])

# Обучение в Keras

In [ ]:
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
print(keras.__version__)

2.2.4


In [ ]:
K.clear_session()  # очищает граф и перезапускает сессию

model = M.Sequential()
model.add(L.Dense(1, use_bias=False, input_shape=(D,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

W0828 05:59:14.830168 140448089249664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
print(x.shape, y.shape)

(1000, 3) (1000, 1)


In [ ]:
model.fit(x, y, batch_size=8, epochs=10, callbacks=[keras.callbacks.TensorBoard("./logs/keras")])

W0828 05:59:19.965025 140448089249664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0828 05:59:19.966285 140448089249664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
1000/1000 [==============================] - 0s 102us/step - loss: 0.7994 - mean_squared_error: 0.7994
Epoch 2/10
1000/1000 [==============================] - 0s 87us/step - loss: 0.1275 - mean_squared_error: 0.1275
Epoch 3/10
1000/1000 [==============================] - 0s 81us/step - loss: 0.0913 - mean_squared_error: 0.0913
Epoch 4/10
1000/1000 [==============================] - 0s 66us/step - loss: 0.0732 - mean_squared_error: 0.0732
Epoch 5/10
1000/1000 [==============================] - 0s 67us/step - loss: 0.0612 - mean_squared_error: 0.0612
Epoch 6/10
1000/1000 [==============================] - 0s 71us/step - loss: 0.0532 - mean_squared_error: 0.0532
Epoch 7/10
1000/1000 [==============================] - 0s 75us/step - loss: 0.0478 - mean_squared_error: 0.0478
Epoch 8/10
1000/1000 [==============================] - 0s 76us/step - loss: 0.0443 - mean_squared_error: 0.0443
Epoch 9/10
1000/1000 [==============================] - 0s 79us/step - loss: 0.0419 - mean_squa

In [ ]:
s = K.get_session()
s.run(model.weights)

[array([[0.757604 ],
        [0.6322889],
        [0.5464471]], dtype=float32)]